In [1]:
pwd

'/Users/akashmurali/Documents/capstone/project/HAR/human-activity-recognition/cnn'

In [11]:
import torch
import numpy as np

# Define model (same as training)
class SimpleCNN(torch.nn.Module):
    def __init__(self, num_classes=10, dropout=0.5):
        super(SimpleCNN, self).__init__()
        
        self.conv1 = torch.nn.Conv1d(3, 64, kernel_size=7, padding=3)
        self.bn1 = torch.nn.BatchNorm1d(64)
        self.pool1 = torch.nn.MaxPool1d(2)
        
        self.conv2 = torch.nn.Conv1d(64, 128, kernel_size=5, padding=2)
        self.bn2 = torch.nn.BatchNorm1d(128)
        self.pool2 = torch.nn.MaxPool1d(2)
        
        self.conv3 = torch.nn.Conv1d(128, 256, kernel_size=3, padding=1)
        self.bn3 = torch.nn.BatchNorm1d(256)
        self.pool3 = torch.nn.MaxPool1d(2)
        
        self.gap = torch.nn.AdaptiveAvgPool1d(1)
        self.dropout = torch.nn.Dropout(dropout)
        self.fc = torch.nn.Linear(256, num_classes)
        
    def forward(self, x):
        x = x.transpose(1, 2)
        
        x = self.pool1(torch.relu(self.bn1(self.conv1(x))))
        x = self.pool2(torch.relu(self.bn2(self.conv2(x))))
        x = self.pool3(torch.relu(self.bn3(self.conv3(x))))
        
        x = self.gap(x).squeeze(-1)
        x = self.dropout(x)
        x = self.fc(x)
        
        return x


# Load model with weights_only=False
print("Loading model...")
checkpoint = torch.load(
    '/Users/akashmurali/Documents/capstone/project/HAR/human-activity-recognition/cnn/best_model.pth',
    map_location='cpu',
    weights_only=False
)

model = SimpleCNN(num_classes=10, dropout=0.5)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

label_encoder = checkpoint['label_encoder']
idx_to_label = {idx: label for label, idx in label_encoder.items()}

print(f"Model loaded")
print(f"Classes: {list(label_encoder.keys())}")
print(f"Val accuracy: {checkpoint.get('val_acc', 'N/A'):.2f}%")

# Load participant data
print("\nLoading data...")
X_all = np.load('/Users/akashmurali/Documents/capstone/project/capture24/preprocessed/X.npy')
Y_all = np.load('/Users/akashmurali/Documents/capstone/project/capture24/preprocessed/Y_WillettsSpecific2018.npy', allow_pickle=True)
P_all = np.load('/Users/akashmurali/Documents/capstone/project/capture24/preprocessed/P.npy', allow_pickle=True)

# Get one participant
participant_mask = P_all == 'P120'
X_test = X_all[participant_mask]
Y_test = Y_all[participant_mask]

print(f"\nParticipant P147:")
print(f"  Windows: {len(X_test)}")
print(f"  Duration: {len(X_test) * 10 / 3600:.2f} hours")

# Predict
print("\nMaking predictions...")
predictions = []
with torch.no_grad():
    for window in X_test:
        window_tensor = torch.FloatTensor(window).unsqueeze(0)
        output = model(window_tensor)
        pred = output.argmax(dim=1).item()
        predictions.append(idx_to_label[pred])

# Calculate accuracy
accuracy = (np.array(predictions) == Y_test).mean() * 100
print(f"\n Accuracy: {accuracy:.2f}%")

# Show some predictions
print("\nSample Predictions:")
print("-" * 50)
for i in range(0, len(predictions), 150):
    match = "✓" if predictions[i] == Y_test[i] else "✗"
    print(f"{match} Window {i}: Predicted={predictions[i]:20s} True={Y_test[i]}")

Loading model...
Model loaded
Classes: [np.str_('bicycling'), np.str_('household-chores'), np.str_('manual-work'), np.str_('mixed-activity'), np.str_('sitting'), np.str_('sleep'), np.str_('sports'), np.str_('standing'), np.str_('vehicle'), np.str_('walking')]
Val accuracy: 75.44%

Loading data...

Participant P147:
  Windows: 5114
  Duration: 14.21 hours

Making predictions...

 Accuracy: 67.23%

Sample Predictions:
--------------------------------------------------
✓ Window 0: Predicted=sleep                True=sleep
✓ Window 150: Predicted=sleep                True=sleep
✓ Window 300: Predicted=sleep                True=sleep
✓ Window 450: Predicted=sleep                True=sleep
✓ Window 600: Predicted=vehicle              True=vehicle
✗ Window 750: Predicted=household-chores     True=sitting
✓ Window 900: Predicted=sitting              True=sitting
✗ Window 1050: Predicted=sleep                True=sitting
✗ Window 1200: Predicted=walking              True=sitting
✓ Window 1350: 